In [49]:
# Import necessary packages
using GLMakie, GeometryBasics, LinearAlgebra, Quaternions, FileIO
import Quaternions: Quaternion as Quaternion

cd(@__DIR__) # Changing the current working directory from project root to the parent directory of this file so that pwd() and @__DIR__ match.
# Import custom code and utility functions
include("../src/State_and_Conversions.jl")
include("../src/Rendering.jl")
include("../src/Transformations.jl")
include("../src/WindowManager.jl")
include("../src/Recorder.jl")



# Initialize the custom structs and utilities
conversions = Conversions()
renderer = Renderer()
transformations = Transformations()
windowmanager = WindowManager()

WindowManager(Main.display_windows, GLMakie.closeall, Makie.inline!)

In [58]:
# Create a state attached scene with a reference state
scene = Scene(camera=cam3d!)
ref = renderer.drawState!(scene)

q = Quaternion(1, 0, 0, 0) # Identity quaternion

state = Observable(State(q))
vectors = [Point3(1.0, 0.0, 0.0), Point3(0.0, 1.0, 0.0), Point3(0.0, 0.0, 1.0)] # Example vectors in the state frame
attached_vectors = renderer.attach2State(vectors, state)
custom_vector_colors = [(1, 0, 0), (0, 1, 0), (0, 0, 1)] # Colors for the custom vectors

for i in eachindex(attached_vectors)
    meshscatter!(scene, (0, 0, 0), marker=lift(p -> Arrow(p).mesh, attached_vectors[i]), markersize=0.75, color=(custom_vector_colors[i]..., 0.5), transparency=true)
end

---
We now have a scene to visualize our rotational states as we perform operations on our quaternion q and update the state to the new quaternion value.

---

## 1. General Information about Quaternions

<img src="..\Notes\Quaternions and Rotations\attachments\Drawing 25-06-09-04-19-25.svg" alt="Quaternion Illustration" style="background: #fff; padding: 10px; border-radius: 8px; display: block; margin: 0 auto; max-width: 100%;">

In [10]:
i = Quaternion(0, 1, 0, 0)
j = Quaternion(0, 0, 1, 0)
k = Quaternion(0, 0, 0, 1)

Quaternion{Int64}(0, 0, 0, 1)

In [19]:
println("i^2 = ", i^2)
println("j^2 = ", j^2)
println("k^2 = ", k^2)
println("i*j*k = ", i*j*k)

i^2 = Quaternion{Int64}(-1, 0, 0, 0)
j^2 = Quaternion{Int64}(-1, 0, 0, 0)
k^2 = Quaternion{Int64}(-1, 0, 0, 0)
i*j*k = Quaternion{Int64}(-1, 0, 0, 0)


In [20]:
println("i*j = ", i*j)
println("j*k = ", j*k)
println("k*i = ", k*i)

i*j = Quaternion{Int64}(0, 0, 0, 1)
j*k = Quaternion{Int64}(0, 1, 0, 0)
k*i = Quaternion{Int64}(0, 0, 1, 0)


## Using Quaternions as rotations

<img src="..\Notes\Quaternions and Rotations\attachments\Drawing 25-06-09-05-28-08.svg" alt="Quaternion Illustration" style="background: #fff; padding: 10px; border-radius: 8px; display: block; margin: 0 auto; max-width: 100%;">

In [59]:
display(scene)

GLMakie.Screen(...)

In [60]:
v = Observable(Point3(1.0, 1.0, 1.0)) # Initial position of the vector
v_mesh = meshscatter!(scene, (0,0,0), markersize=1, marker=lift(v-> Arrow(normalize(v)).mesh, v), color=(1, 1, 1, 0.5), transparency=true)

MeshScatter{Tuple{Vector{Point{3, Float64}}}}

In [62]:
transformations.interpolate_vector(v, [1,0,0]; rate_function=t -> (1 - cos(t * π)) / 2) # Transform to [1, 0, 0] with a smooth transition
sleep(2) # Wait for the transition to complete
transformations.interpolate_vector(v, [1,1,1]; rate_function=t -> (1 - cos(t * π)) / 2) # Transform back to [1, 1, 1] with a smooth transition

#### Showcase quaternions as a means to rotate a vector

In [75]:
e=ℯ # Euler's number

n_hat = i
theta = π / 4 # Rotation angle in radians
q_rot = e^(theta / 2 * n_hat) # Quaternion representing the rotation


println(q_rot)
println(inv(q_rot))

QuaternionF64(0.9238795325112867, 0.3826834323650898, 0.0, 0.0)
QuaternionF64(0.9238795325112867, -0.3826834323650898, -0.0, -0.0)


In [87]:
println(v[])

# Apply the rotation to the vector
v_rotated = imag_part(q_rot * Quaternion(0, v[]...) * inv(q_rot))
println(v_rotated)

[1.0000000000000002, -1.4142135623730951, -5.551115123125783e-16]
(1.0000000000000002, -0.9999999999999996, -1.0000000000000004)


In [88]:
transformations.interpolate_vector(v, Point3(v_rotated); rate_function=t -> (1 - cos(t * π)) / 2) # Smoothly rotate the vector

In [89]:
v[] = Point3(1.0, 1.0, 1.0) # Reset the vector position

3-element Point{3, Float64} with indices SOneTo(3):
 1.0
 1.0
 1.0

In [90]:
delete!(scene, v_mesh) # Remove the previous vector mesh

Using this principal of Quaternions describing rotation of a vector about an axis, for a given angle, we can thus define rotational state for an entire 3D object using a quaternion, where all points within the object would be rotated by the quaternion `q_state` at any given point in time

In [91]:
q, state

(Quaternion{Int64}(1, 0, 0, 0), Observable(State([1.0, 0.0, 0.0], [0.0, 1.0, 0.0], [0.0, 0.0, 1.0], QuaternionF64(1.0, 0.0, 0.0, 0.0))))

In [93]:
q = e^(π / 4 * i) # Quaternion representing a rotation around the x-axis by 90 degrees
transformations.interpolate_states(state, State(q); rate_function=t -> (1 - cos(t * π)) / 2) # Smoothly transition the state to the new quaternion

In [95]:
q = conversions.axisangle2quat([1,0,0], π / 4) # Convert axis-angle representation to quaternion
transformations.interpolate_states(state, State(q); rate_function=t -> (1 - cos(t * π)) / 2) # Smoothly transition the state to the new quaternion

In [99]:
q = Quaternion(1, 0, 0, 0) # Reset to identity quaternion
transformations.interpolate_states(state, State(q); rate_function=t -> (1 - cos(t * π)) / 2) # Smoothly transition back to the identity quaternion

#### Combining Rotational Operations

Mathematically,
$$q2*q1=q$$

would be interpretet as **$q$ is the quaternion that gives us the net rotation which if applied is equivalent to first applying $q1$, then applying $q2$**

(note... left multiplying is the order in which we state global rotational operations... similar to Matrices)

In [100]:
# Apply 1st rotation
q1 = conversions.axisangle2quat([1, 0, 0], π / 2) # 90 degrees around x-axis
transformations.interpolate_states(state, State(q1); rate_function=t -> (1 - cos(t * π)) / 2) # Smoothly transition to the first rotation

# Apply 2nd rotation
q2 = conversions.axisangle2quat([0, 1, 0], π / 2) # 90 degrees around y-axis
q = q2*q1 # First apply q1, then q2
transformations.interpolate_states(state, State(q); rate_function=t -> (1 - cos(t * π)) / 2) # Smoothly transition to the second rotation

## Angular Velocity

In [107]:
axis = normalize([1, 1, 1]) # Rotation axis
omega = pi # radians per second
dt = 1/30 # 30 FPS
q_rot = conversions.axisangle2quat([1, 0, 0], omega * dt) # Small rotation around z-axis

for i in 1:30
    q = q_rot * state[].q # Apply the small rotation
    state[] = State(q) # Update the state
    sleep(dt) # Wait for the next frame
end

## SLERP: How to know intermediate states?

```julia
function interpolate_states(from::Observable{State}, to::State; n::Int=100, time::Real=1.0, rate_function=t -> t)
    # Interpolate between two states from_copy to to with n steps and time duration time
    from_copy = from[] # Copy the initial state
    for i in 1:n
        t = i / n
        from[] = State(slerp(from_copy.q, to.q, rate_function(t)))
        sleep(time / n) # Sleep for a short duration to control the speed of the animation
        yield()
    end
end
```
This is the function definition I use to animate from State 1 to State 2.

Notice that all I use is the function `slerp(q1, q2, t)`

This SLERP function (or Spherical Linear Interpolation) gives you the intermediate quaternion from q1 and q2, for given alpha value t, which ranges from 0 to 1

### How does the `slerp` function work for Quaternions?

The **Spherical Linear Interpolation (SLERP)** function computes a smooth interpolation between two unit quaternions, `q1` and `q2`, parameterized by `t ∈ [0, 1]`. This is essential for smoothly animating rotations in 3D space.

#### Mathematical Principle

Given two unit quaternions `q1` and `q2`, the SLERP formula is:

$$
\text{slerp}(q_1, q_2, t) = \frac{\sin((1-t)\theta)}{\sin\theta} q_1 + \frac{\sin(t\theta)}{\sin\theta} q_2
$$

where $\theta$ is the angle between `q1` and `q2` in quaternion space, computed as:

$$
\cos\theta = \langle q_1, q_2 \rangle
$$

This formula ensures constant angular velocity and the shortest path on the 4D unit sphere.

#### Defining Our Own SLERP Function

Suppose we want to transform `q1` to `q2` by some operation $q_\text{star}$:

- By definition, if $q_\text{star} * q1 = q2$, then $q_\text{star} = q2 \cdot q1^{-1}$.
- Therefore, the intermediate quaternion at time `t` can be written as:

$$
q(t) = (q_2 q_1^{-1})^t \cdot q_1
$$

- Here, exponentiating the quaternion smoothly scales the rotation angle about the same axis.

To understand this process step by step:

1. **Find the relative rotation:**  
    Compute $q_\text{star} = q2 \cdot q1^{-1}$.

2. **Extract axis and angle:**  
    Use $quat2axisangle(q_\text{star})$ to get the rotation axis and angle.

3. **Define incremental rotation:**  
    For $n$ steps, define a small angle increment $d\theta$ such that $n \cdot d\theta = \text{total angle}$.

4. **Apply incremental rotations:**  
    At each step, create a quaternion for the small rotation:  
    `dq_star = quat2axisangle(axis, dtheta)`

5. **Relate to angular velocity:**  
    The angular velocity $\omega$ relates to the increment as $\omega \cdot dt = d\theta$.

6. **Iterate:**  
    Applying this incremental quaternion $n$ times should take you from `q1` to `q2`.

---

In [108]:
function custom_slerp_incremental(q1::Quaternion, q2::Quaternion, n::Int)
    """
    Custom SLERP function that returns the incremental quaternion dq_star
    which when applied n times takes you from q1 to q2.
    
    Returns:
    - q_star: The total relative rotation quaternion
    - axis: The rotation axis
    - angle: The total rotation angle
    - dq_star: The incremental quaternion to apply n times
    """
    
    # Step 1: Find the relative rotation
    q_star = q2 * inv(q1)
    
    # Step 2: Extract axis and angle from q_star
    axis, angle = conversions.quat2axisangle(q_star)
    
    # Handle the case where there's no rotation (angle ≈ 0)
    if abs(angle) < 1e-10
        return q_star, [1.0, 0.0, 0.0], 0.0, Quaternion(1, 0, 0, 0)
    end
    
    # Step 3: Define incremental rotation
    dtheta = angle / n  # Small angle increment
    
    # Step 4: Create the incremental quaternion
    dq_star = conversions.axisangle2quat(axis, dtheta)
    
    return q_star, axis, angle, dq_star
end

custom_slerp_incremental (generic function with 1 method)

In [113]:
# Get the incremental quaternion manually
q_current = state[].q
q_target = conversions.axisangle2quat([1, 0, 0], π/2)  # 90 degrees around z-axis

q_star, axis, angle, dq_star = custom_slerp_incremental(q_current, q_target, 30)

println("Apply this quaternion 30 times: ", dq_star)
println("Total rotation: $(rad2deg(angle)) degrees around axis: $(axis)")

# Apply it manually like your example
for i in 1:30
    q = dq_star * state[].q  # Apply the small rotation
    state[] = State(q)
    sleep(1/30)
end

Apply this quaternion 30 times: QuaternionF64(0.9996573249755573, 0.026176948307873142, 1.744200366741944e-33, 1.5702687250760503e-33)
Total rotation: 89.99999999999997 degrees around axis: [1.0, 6.663115754472218e-32, 5.998669923658621e-32]
